In [ ]:
import obsidian
print(f'obsidian version: ' + obsidian.__version__)

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

## Set up parameter space and initialize a design

In [ ]:
from obsidian import Campaign, Target, ParamSpace, BayesianOptimizer
from obsidian.parameters import Param_Categorical, Param_Ordinal, Param_Continuous

In [ ]:
params = [
    Param_Continuous('Temperature', -10, 30),
    Param_Continuous('Concentration', 10, 150),
    Param_Continuous('Enzyme', 0.01, 0.30),
    Param_Categorical('Variant', ['MRK001', 'MRK002', 'MRK003']),
    Param_Ordinal('Stir Rate', ['Low', 'Medium', 'High']),
    ]

X_space = ParamSpace(params)
target = Target('Yield', aim='max')
campaign = Campaign(X_space, target, seed=0)
X0 = campaign.initialize(m_initial = 10, method = 'LHS')

X0

## Collect results (e.g. from a simulation)

In [ ]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import shifted_parab

simulator = Simulator(X_space, shifted_parab, name='Yield', eps=0.05)
y0 = simulator.simulate(X0)
Z0 = pd.concat([X0, y0], axis=1)

campaign.add_data(Z0)
campaign.data.sample(5)

In [ ]:
from obsidian.plotting import visualize_inputs

fig = visualize_inputs(campaign)

## Fit the optimizer and visualize results

In [ ]:
campaign.fit()

In [ ]:
from obsidian.plotting import parity_plot, factor_plot, optim_progress

In [ ]:
parity_plot(campaign.optimizer)

In [ ]:
factor_plot(campaign.optimizer, feature_id=0)

## Optimize new experiment suggestions

In [ ]:
X_suggest, eval_suggest = campaign.suggest(m_batch=3)

In [ ]:
df_suggest = pd.concat([X_suggest, eval_suggest], axis=1)
df_suggest

## Collect data at new suggestions

In [ ]:
y_iter1 = pd.DataFrame(simulator.simulate(X_suggest), columns = ['Yield'])
Z_iter1 = pd.concat([X_suggest, y_iter1, eval_suggest], axis=1)
campaign.add_data(Z_iter1)
campaign.data.tail()

## Repeat as desired

In [ ]:
for iter in range(3):
    campaign.fit()
    X_suggest, eval_suggest = campaign.suggest(m_batch=3)
    y_iter = pd.DataFrame(simulator.simulate(X_suggest))
    Z_iter = pd.concat([X_suggest, y_iter, eval_suggest], axis=1)
    campaign.add_data(Z_iter)

In [ ]:
optim_progress(campaign, color_feature_id = 'aq Value')

## Analyze using Explainer

In [ ]:
from obsidian.campaign import Explainer

In [ ]:
exp = Explainer(campaign.optimizer)
exp.shap_explain(n=500)

In [ ]:
exp.shap_summary()

In [ ]:
exp.shap_pdp_ice(ind = 0, ice_color_var = 2)